In [1]:
import pandas as pd
import numpy as np
import math 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
data_train = open("data_train.txt")
label_train = open("labels_train_original.txt")
data_test = open("data_valid.txt")
label_test = open("labels_valid_original.txt")

# 1. Text Preprocessing 
In order to perform machine learning on text documents, we first need to turn the text content into numerical feature vectors.

## 1.1 replace text labels in training set to numerical labels

In [2]:
replace = {
    'News':0,
    'Classifieds':2,
    'Opinion':1,
    'Features':3
}
Y_train = [replace[y.rstrip('\n')] for y in label_train]

## 1.2 Tokenizing the artical in training set
CountVectorizer() builds a dictionary of features and transforms documents to feature vectors.
The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.

In [3]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data_train)
X_train_counts.shape

(2000, 38749)

In [13]:
dict(list(count_vect.vocabulary_.items())[0:5])

{'front': 14711, 'in': 17827, 'of': 24546, 'sign': 31683, 'the': 34753}

## 1.3 Term Frequency times Inverse Document Frequency
The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus.

In [14]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2000, 38749)

In [21]:
type(X_train_tfidf)

scipy.sparse.csr.csr_matrix

## 1.4 Do the same things to the test set

In [642]:
Y_test = [replace[y.rstrip('\n')] for y in label_test]

In [643]:
X_test = list()
for line in data_test:
    line = str(line.rstrip())
    X_test.append(line)

In [644]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# 2. Classifications

## 2.1 Naive Bayes

In [675]:
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB(alpha=0.6).fit(X_train_tfidf, Y_train)
clf_NB

MultinomialNB(alpha=0.6, class_prior=None, fit_prior=True)

In [676]:
NB_train_predicted = clf_NB.predict(X_train_tfidf)

In [677]:
NB_train_CM = confusion_matrix(Y_train, NB_train_predicted)
print('Confusion Matrix of Training Set:\n',NB_train_CM)
print('Accuracy Rate of Training Set:\n',
      (NB_train_CM[0][0]+NB_train_CM[1][1]+NB_train_CM[2][2]+NB_train_CM[3][3])/2000)

Confusion Matrix of Training Set:
 [[466  13   9  16]
 [  9 465   4   7]
 [ 19   7 460  37]
 [ 22  26   1 439]]
Accuracy Rate of Training Set:
 0.915


In [678]:
NB_test_predicted = clf_NB.predict(X_test_tfidf)

In [679]:
NB_test_CM = confusion_matrix(Y_test, NB_test_predicted)
print('Confusion Matrix of Test Set:\n',NB_test_CM)
print('Accuracy Rate of Test Set:\n',(NB_test_CM[0][0]+NB_test_CM[1][1]+NB_test_CM[2][2]+NB_test_CM[3][3])/2000)

Confusion Matrix of Test Set:
 [[314  40  65  93]
 [ 25 414  16  52]
 [ 58  22 297  93]
 [ 52  82  80 297]]
Accuracy Rate of Test Set:
 0.661


## 2.2 Logistic Regression 

In [258]:
from sklearn.linear_model import LogisticRegression
clf_LR = LogisticRegression(random_state=0,C=75).fit(X_train_tfidf, Y_train)
clf_LR

LogisticRegression(C=75, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [259]:
LR_train_predicted = clf_LR.predict(X_train_tfidf)

In [260]:
LR_train_CM = confusion_matrix(Y_train, LR_train_predicted)
print('Confusion Matrix of Training Set:\n',LR_train_CM)
print('Accuracy Rate of Training Set:\n',
      (LR_train_CM[0][0]+LR_train_CM[1][1]+LR_train_CM[2][2]+LR_train_CM[3][3])/2000)

Confusion Matrix of Training Set:
 [[504   0   0   0]
 [  0 485   0   0]
 [  0   0 523   0]
 [  0   0   0 488]]
Accuracy Rate of Training Set:
 1.0


In [261]:
LR_test_predicted = clf_LR.predict(X_test_tfidf)

In [262]:
LR_test_CM = confusion_matrix(Y_test, LR_test_predicted)
print('Confusion Matrix of Test Set:\n',LR_test_CM)
print('Accuracy Rate of Test Set:\n',(LR_test_CM[0][0]+LR_test_CM[1][1]+LR_test_CM[2][2]+LR_test_CM[3][3])/2000)

Confusion Matrix of Test Set:
 [[328  41  67  76]
 [ 16 438  20  33]
 [ 41  14 350  65]
 [ 60  77  71 303]]
Accuracy Rate of Test Set:
 0.7095


## 2.3 SVM ( SVC with Linear Kernel)

In [497]:
from sklearn import svm
clf_svm1 = svm.SVC(decision_function_shape='ovo',kernel='linear',C=2).fit(X_train_tfidf, Y_train)
clf_svm1

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [498]:
svm1_train_predicted = clf_svm1.predict(X_train_tfidf)

In [499]:
svm1_train_CM = confusion_matrix(Y_train, svm1_train_predicted)
print('Confusion Matrix of Training Set:\n',svm1_train_CM)
print('Accuracy Rate of Training Set:\n',
      (svm1_train_CM[0][0]+svm1_train_CM[1][1]+svm1_train_CM[2][2]+svm1_train_CM[3][3])/2000)

Confusion Matrix of Training Set:
 [[503   0   1   0]
 [  0 485   0   0]
 [  0   0 523   0]
 [  3   0   0 485]]
Accuracy Rate of Training Set:
 0.998


In [500]:
svm1_test_predicted = clf_svm1.predict(X_test_tfidf)

In [501]:
svm1_test_CM = confusion_matrix(Y_test, svm1_test_predicted)
print('Confusion Matrix of Test Set:\n',svm1_test_CM)
print('Accuracy Rate of Test Set:\n',(svm1_test_CM[0][0]+svm1_test_CM[1][1]+svm1_test_CM[2][2]+svm1_test_CM[3][3])/2000)

Confusion Matrix of Test Set:
 [[324  37  67  84]
 [ 15 435  20  37]
 [ 44  13 343  70]
 [ 57  69  70 315]]
Accuracy Rate of Test Set:
 0.7085


## 2.4 SVM ( Linear SVC)

### Similar to SVC with parameter kernel=’linear’, but implemented in terms of liblinear rather than libsvm, so it has more flexibility in the choice of penalties and loss functions and should scale better to large numbers of samples.

In [532]:
clf_svm2 = svm.LinearSVC().fit(X_train_tfidf, Y_train)
clf_svm2

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [533]:
svm2_train_predicted = clf_svm2.predict(X_train_tfidf)

In [534]:
svm2_train_CM = confusion_matrix(Y_train, svm2_train_predicted)
print('Confusion Matrix of Training Set:\n',svm2_train_CM)
print('Accuracy Rate of Training Set:\n',
      (svm2_train_CM[0][0]+svm2_train_CM[1][1]+svm2_train_CM[2][2]+svm2_train_CM[3][3])/2000)

Confusion Matrix of Training Set:
 [[504   0   0   0]
 [  0 485   0   0]
 [  0   0 523   0]
 [  1   0   0 487]]
Accuracy Rate of Training Set:
 0.9995


In [535]:
svm2_test_predicted = clf_svm2.predict(X_test_tfidf)

In [536]:
svm2_test_CM = confusion_matrix(Y_test, svm2_test_predicted)
print('Confusion Matrix of Test Set:\n',svm2_test_CM)
print('Accuracy Rate of Test Set:\n',(svm2_test_CM[0][0]+svm2_test_CM[1][1]+svm2_test_CM[2][2]+svm2_test_CM[3][3])/2000)

Confusion Matrix of Test Set:
 [[331  43  65  73]
 [ 15 443  19  30]
 [ 42  17 341  70]
 [ 55  81  71 304]]
Accuracy Rate of Test Set:
 0.7095


## 2.5 KNN

In [633]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=15,weights='distance').fit(X_train_tfidf, Y_train)

In [634]:
neigh_predicted = neigh.predict(X_train_tfidf)

In [635]:
neigh_train_CM = confusion_matrix(Y_train, neigh_predicted)
print('Confusion Matrix of Training Set:\n',neigh_train_CM)
print('Accuracy Rate of Training Set:\n',
      (neigh_train_CM[0][0]+neigh_train_CM[1][1]+neigh_train_CM[2][2]+neigh_train_CM[3][3])/2000)

Confusion Matrix of Training Set:
 [[504   0   0   0]
 [  0 485   0   0]
 [  0   0 523   0]
 [  0   0   0 488]]
Accuracy Rate of Training Set:
 1.0


In [636]:
neigh_test_predicted = neigh.predict(X_test_tfidf)

In [637]:
neigh_test_CM = confusion_matrix(Y_test, neigh_test_predicted)
print('Confusion Matrix of Test Set:\n',neigh_test_CM)
print('Accuracy Rate of Test Set:\n',
      (neigh_test_CM[0][0]+neigh_test_CM[1][1]+neigh_test_CM[2][2]+neigh_test_CM[3][3])/2000)

Confusion Matrix of Test Set:
 [[299  31 135  47]
 [ 38 274 127  68]
 [ 43  19 365  43]
 [ 78  36 177 220]]
Accuracy Rate of Test Set:
 0.579
